In [1]:
!which python

/u/pw7nc/anaconda3/bin/python


In [2]:
import re
import json
import os

import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer

import spacy
import nltk
from nltk.tokenize import sent_tokenize
from spacy.lang.en import English
nlp = English()
# Create a Tokenizer with the default settings for English
# including punctuation rules and exceptions
tokenizer = nlp.tokenizer
import string
punct = string.punctuation
from sklearn.feature_extraction import _stop_words

In [3]:
dataset_name = "yelp"

# Load Data

## Load Dataset

In [4]:
dir_path = '../Dataset/{}'.format(dataset_name)
# Load train dataset
train_review = []
cnt = 0
file_path = os.path.join(dir_path, 'train_review_filtered.json')
with open(file_path) as f:
    print("Load file: {}".format(file_path))
    for line in f:
        line_data = json.loads(line)
        user_id = line_data['user']
        item_id = line_data['item']
        rating = line_data['rating']
        review = line_data['review']
        train_review.append([item_id, user_id, rating, review])
        cnt += 1
        if cnt % 100000 == 0:
            print('{} lines loaded.'.format(cnt))
print('Finish loading train dataset, totally {} lines.'.format(len(train_review)))
# Load test dataset
test_review = []
cnt = 0
file_path = os.path.join(dir_path, 'test_review_filtered_clean.json')
with open(file_path) as f:
    print("Load file: {}".format(file_path))
    for line in f:
        line_data = json.loads(line)
        user_id = line_data['user']
        item_id = line_data['item']
        rating = line_data['rating']
        review = line_data['review']
        test_review.append([item_id, user_id, rating, review])
        cnt += 1
        if cnt % 10000 == 0:
            print('{} lines loaded.'.format(cnt))
print('Finish loading test dataset, totally {} lines.'.format(len(test_review)))

Load file: ../Dataset/yelp/train_review_filtered.json
100000 lines loaded.
Finish loading train dataset, totally 191227 lines.
Load file: ../Dataset/yelp/test_review_filtered_clean.json
10000 lines loaded.
20000 lines loaded.
30000 lines loaded.
40000 lines loaded.
Finish loading test dataset, totally 42702 lines.


In [5]:
df_train_data = pd.DataFrame(train_review, columns=['item', 'user', 'rating', 'review'])
df_test_data = pd.DataFrame(test_review, columns=['item', 'user', 'rating', 'review'])

In [6]:
df_train_data

,item,user,rating,review
0,1071,1001,4,"place was well lit , very clean , and staff ve..."
1,10877,1001,3,the price was great and so was the subs ... th...
2,1114,1001,3,previous i was bundled into a takeout work ord...
3,120,1001,5,this is my favorite restaurant by far . food o...
4,1278,1001,4,"nice little gem right here . ok , staff were v..."
...,...,...,...,...
191222,4993,9999,5,my two favorites are the carne asada and barba...
191223,704,9999,4,the tacos themselves were a mixed bag . in par...
191224,7379,9999,5,everything was great .
191225,8530,9999,4,our last experience here went pretty well and ...


In [7]:
# groupby multiple columns
groupby_user_item = df_train_data.groupby(['user', 'item'])
cnt = 0
for key, item in groupby_user_item:
    cur_df_user_item = groupby_user_item.get_group(key)
    if len(cur_df_user_item) > 1:
        if cnt <= 10:
            print(cur_df_user_item)
        cnt += 1
print("{} data instance are the same".format(cnt))
# make sure that there are no duplicated reviews

0 data instance are the same


## Load Sentence2ID and ID2Sentence Mapping

In [8]:
sentence2id_filepath = '../Dataset/{}/train/sentence/sentence2id.json'.format(dataset_name)
with open(sentence2id_filepath, 'r') as f:
    print("Load file: {}".format(sentence2id_filepath))
    sent_to_id = json.load(f)

Load file: ../Dataset/yelp/train/sentence/sentence2id.json


In [9]:
type(sent_to_id['good service .'])

str

In [10]:
sent_to_id['good service .']

'13'

In [11]:
id2sentence_filepath = '../Dataset/{}/train/sentence/id2sentence.json'.format(dataset_name)
with open(id2sentence_filepath, 'r') as f:
    print("Load file: {}".format(id2sentence_filepath))
    id_to_sent = json.load(f)

Load file: ../Dataset/yelp/train/sentence/id2sentence.json


In [12]:
id_to_sent['0']

'service was good .'

In [13]:
assert len(sent_to_id) == len(id_to_sent)
print("Number of sentence (with feature) on train set: {}".format(len(sent_to_id)))

Number of sentence (with feature) on train set: 492739


## Load Feature Words

In [14]:
feature2id_filepath = '../Dataset/{}/train/feature/feature2id.json'.format(dataset_name)
with open(feature2id_filepath, 'r') as f:
    print("Load file: {}".format(feature2id_filepath))
    feature_vocab = json.load(f)

Load file: ../Dataset/yelp/train/feature/feature2id.json


In [15]:
len(feature_vocab)

498

In [16]:
feature_vocab['wifi']

'371'

## Load Sentence2Feature

In [17]:
sentence2feature_filepath = '../Dataset/{}/train/sentence/sentence2feature.json'.format(dataset_name)
with open(sentence2feature_filepath, 'r') as f:
    print("Load file: {}".format(sentence2feature_filepath))
    sentence_to_feature = json.load(f)

Load file: ../Dataset/yelp/train/sentence/sentence2feature.json


In [18]:
sentence_to_feature['0']

{'1': 1.0}

In [19]:
assert len(sentence_to_feature) == len(sent_to_id)
len(sentence_to_feature)

492739

# Construct User-Item Pair

## GrounpBy User

In [20]:
group_by_user = df_train_data.groupby('user')
group_by_user_dict = dict(tuple(group_by_user))

In [22]:
group_by_user_dict['1001']

,item,user,rating,review
0,1071,1001,4,"place was well lit , very clean , and staff ve..."
1,10877,1001,3,the price was great and so was the subs ... th...
2,1114,1001,3,previous i was bundled into a takeout work ord...
3,120,1001,5,this is my favorite restaurant by far . food o...
4,1278,1001,4,"nice little gem right here . ok , staff were v..."
...,...,...,...,...
81,9121,1001,4,for a waffle house this place is a little bett...
82,9203,1001,1,"++ the chips were good , salsa eh, , ,"
83,9307,1001,4,"food was very good and great price , ,"
84,9507,1001,5,"awesome menu options , large sized menu is pre..."


In [23]:
len(group_by_user_dict)

4604

In [24]:
user_id_list = list(df_train_data['user'].unique())

In [25]:
print(len(user_id_list))

4604


In [26]:
user_to_sent = dict()
cnt_sentence_with_no_feature = 0

for user_id in user_id_list:
    # note this user_id is a str
    # get the dataframe for this user
    user_df = group_by_user_dict[user_id]
    user_reviews = list(user_df['review'])
    user_sent_ids = set()
    for review in user_reviews:
        # tokenize this review (i.e. split into sentences)
        review_sents = sent_tokenize(review)
        # check whether the sentence is in the sentence2id dictionary
        for sent in review_sents:
            if sent in sent_to_id:
                cur_sent_id = sent_to_id[sent]
                assert isinstance(cur_sent_id, str)
                # make sure that this sentence has feature
                # assert cur_sent_id in sentence_to_feature
                # user_sent_ids.add(cur_sent_id)
                if cur_sent_id in sentence_to_feature:
                    # add this sentence into the set of this user
                    user_sent_ids.add(cur_sent_id)
                else:
                    cnt_sentence_with_no_feature += 1
        # TODO: what should we do if there are sentence that appears in multiple reivews?
    if len(user_sent_ids) == 0:
        print("User: {} has no effective sentences, skip it.".format(user_id))
    else:
        user_to_sent[user_id] = user_sent_ids
print('{} sentence with no feature'.format(cnt_sentence_with_no_feature))

0 sentence with no feature


In [27]:
len(user_to_sent)

4604

In [28]:
user_to_sentids = dict()
for user_id, user_sents in user_to_sent.items():
    assert len(user_sents) > 0
    assert isinstance(user_id, str)
    assert isinstance(list(user_sents)[0], str)
    user_to_sentids[user_id] = list(user_sents)

In [29]:
len(user_to_sentids)

4604

In [30]:
user2sentids_filepath = '../Dataset/{}/train/user/user2sentids.json'.format(dataset_name)
with open(user2sentids_filepath, 'w') as f:
    print("Write file: {}".format(user2sentids_filepath))
    json.dump(user_to_sentids, f)

Write file: ../Dataset/yelp/train/user/user2sentids.json


In [31]:
user_side_cdd_sents_num = list()
for key, value in user_to_sentids.items():
    user_side_cdd_sents_num.append(len(value))

In [32]:
print("Mean number of sentence per user: {}".format(
    np.mean(user_side_cdd_sents_num)
))
print("Min number of sentence per user: {}".format(
    np.min(user_side_cdd_sents_num)
))
print("Max number of sentence per user: {}".format(
    np.max(user_side_cdd_sents_num)
))

Mean number of sentence per user: 111.28996524761077
Min number of sentence per user: 18
Max number of sentence per user: 1756


In [33]:
print("Top-10 least numbber of sentences per user: {}".format(
    sorted(user_side_cdd_sents_num)[:10]
))
print("Top-10 most numbber of sentences per user: {}".format(
    sorted(user_side_cdd_sents_num)[-10:]
))

Top-10 least numbber of sentences per user: [18, 18, 19, 20, 21, 21, 21, 21, 21, 22]
Top-10 most numbber of sentences per user: [946, 970, 999, 1000, 1037, 1074, 1274, 1421, 1550, 1756]


## GroupBy Item

In [34]:
group_by_item = df_train_data.groupby('item')
group_by_item_dict = dict(tuple(group_by_item))

In [35]:
group_by_item_dict['1267']

,item,user,rating,review
7006,1267,1103,5,service was quick and very attentive . the muf...
8846,1267,11259,5,dineen coffee co is a sleek and modern coffee ...
23902,1267,1376,4,the pistachio bread was surprisingly soft like...
25199,1267,140,4,"coffee was amazing , by amazing i mean i would..."
28184,1267,147,4,the staff are very nice and very accommodating...
30670,1267,1519,5,nice patio . limited outdoor seating .
31802,1267,1547,4,"their espresso blend was delicious , strong bu..."
32245,1267,1564,4,my coffee was as high quality as is to be expe...
46910,1267,196,3,downtown toronto on a sunday morning is a ghos...
47230,1267,1967,3,but the atmosphere is lovely . cute spot for c...


In [36]:
len(group_by_item_dict)

7837

In [37]:
item_id_list = list(df_train_data['item'].unique())
item_to_sent = dict()
cnt_sentence_with_no_feature = 0
for item_id in item_id_list:
    # note this item_id is a str
    # get the dataframe for this item
    assert isinstance(item_id, str)
    item_df = group_by_item_dict[item_id]
    item_reviews = list(item_df['review'])
    item_sent_ids = set()
    for review in item_reviews:
        # tokenize this review (i.e. split into sentences)
        review_sents = sent_tokenize(review)
        # check whether the sentence is in the sentence2id dictionary
        for sent in review_sents:
            if sent in sent_to_id:
                cur_sent_id = sent_to_id[sent]
                assert isinstance(cur_sent_id, str)
                # make sure that this sentence has feature
                if cur_sent_id in sentence_to_feature:
                    # add this sentence into the set of this user
                    item_sent_ids.add(cur_sent_id)
                else:
                    cnt_sentence_with_no_feature += 1
        # TODO: what should we do if there are sentence that appears in multiple reivews?
    if len(item_sent_ids) == 0:
        print("Item {} has no effective sentence, skip it.".format(item_id))
    else:
        item_to_sent[item_id] = item_sent_ids

print('{} sentence with no feature'.format(cnt_sentence_with_no_feature))

0 sentence with no feature


In [38]:
len(item_to_sent)

7837

In [39]:
item_to_sentids = dict()
for item_id, item_sents in item_to_sent.items():
    assert len(item_sents) > 0
    assert isinstance(list(item_sents)[0], str)
    item_to_sentids[item_id] = list(item_sents)

In [40]:
len(item_to_sentids)

7837

In [41]:
item2sentids_filepath = '../Dataset/{}/train/item/item2sentids.json'.format(dataset_name)
with open(item2sentids_filepath, 'w') as f:
    print("Write file: {}".format(item2sentids_filepath))
    json.dump(item_to_sentids, f)

Write file: ../Dataset/yelp/train/item/item2sentids.json


In [42]:
item_side_cdd_sents_num = list()
for key, value in item_to_sentids.items():
    item_side_cdd_sents_num.append(len(value))

In [43]:
print("Mean number of sentence per item: {}".format(
    np.mean(item_side_cdd_sents_num)
))
print("Min number of sentence per item: {}".format(
    np.min(item_side_cdd_sents_num)
))
print("Max number of sentence per item: {}".format(
    np.max(item_side_cdd_sents_num)
))

Mean number of sentence per item: 65.58683169580196
Min number of sentence per item: 5
Max number of sentence per item: 625


In [44]:
print("Top-10 least numbber of sentence per item: {}".format(
    sorted(item_side_cdd_sents_num)[:10]
))
print("Top-10 most numbber of sentence per item: {}".format(
    sorted(item_side_cdd_sents_num)[-10:]
))

Top-10 least numbber of sentence per item: [5, 7, 7, 7, 8, 8, 8, 8, 8, 8]
Top-10 most numbber of sentence per item: [455, 457, 457, 471, 473, 512, 540, 550, 579, 625]


# For Each Data Instance in TrainSet

In [45]:
import random
sample_sent_num = 500
user_item_candidate_sent_ids = dict()
# Loop over all User
user_cnt = 0
review_cnt = 0
review_with_no_selectd_label_sentence = 0
useable_review_cnt = 0
sentence_with_no_feature_cnt = 0
sentence_not_tracked = set()
for user_df_chunk in list(group_by_user):
    user_id = int(user_df_chunk[0])
    user_id_str = str(user_df_chunk[0])
    user_df = user_df_chunk[1]
    # get user sents
    cur_user_sent_ids = user_to_sent[user_id_str]
    # item-level dict
    item_candidate_sent_ids = dict()
    for idx, row in user_df.iterrows():
        item_id = int(row['item'])
        item_id_str = str(row['item'])
        review_text = row['review']
        review_cnt += 1
        # get item sents
        cur_item_sent_ids = item_to_sent[item_id_str]
        # get review_text's sent ids
        cur_review_sent_ids = set()
        ## tokenize this review
        review_sents = sent_tokenize(review_text)
        ## check whether this sentence is in the sentence2id dictionary
        for sent in review_sents:
            if sent in sent_to_id:
                cur_sent_id = sent_to_id[sent]
                assert isinstance(cur_sent_id, str)
                # make sure that this sentence has feature
                if cur_sent_id in sentence_to_feature:
                    # add this sentence into the set of current review
                    cur_review_sent_ids.add(cur_sent_id)
                else:
                    sentence_with_no_feature_cnt += 1
            else:
                # if this sentence is not being tracked by the sentence-id mapping
                # we add it into this set to see how many sentences are being ignored
                sentence_not_tracked.add(sent)
        ## check whether the true label of the sentence is an empty list of sent_ids
        if len(cur_review_sent_ids) == 0:
            review_with_no_selectd_label_sentence += 1
        else:
            # construct the candidate set which is an union of user sentences and item sentences
            cur_useritem_sent_ids = cur_user_sent_ids | cur_item_sent_ids
            # sample some sentences
            if len(cur_useritem_sent_ids) > sample_sent_num:
                sample_useritem_sent_ids = set(random.sample(cur_useritem_sent_ids, sample_sent_num))
            else:
                sample_useritem_sent_ids = cur_useritem_sent_ids
            # union sampled sentences with true labeled sentences
            final_useritem_sent_ids = sample_useritem_sent_ids | cur_review_sent_ids
            # add this into the dict
            item_candidate_sent_ids[item_id_str] = [list(final_useritem_sent_ids), list(cur_review_sent_ids)]
            # add useable review cnt
            useable_review_cnt += 1
    if len(item_candidate_sent_ids) == 0:
        print("User: {} has no useful item, skip this user ...".format(user_id_str))
    else:
        # add the item_candidate_sent_ids dict into the user-level dict
        user_item_candidate_sent_ids[user_id_str] = item_candidate_sent_ids
    user_cnt += 1
    if user_cnt % 1000 == 0:
        print("{} user processed".format(user_cnt))

print('Finish.')
print('Totally {} users'.format(user_cnt))
print('Totally {0} reviews. Among them {1} reviews has empty true label sentence'.format(
    review_cnt, review_with_no_selectd_label_sentence))
print("{} sentences has 0 feature".format(sentence_with_no_feature_cnt))
print("{} sentences are not being tracked in the sent2id mapping".format(len(sentence_not_tracked)))

1000 user processed
2000 user processed
3000 user processed
4000 user processed
Finish.
Totally 4604 users
Totally 191227 reviews. Among them 0 reviews has empty true label sentence
0 sentences has 0 feature
929 sentences are not being tracked in the sent2id mapping


In [46]:
len(user_item_candidate_sent_ids)

4604

In [47]:
# let's check how many unique reviews are there

cnt_unique_reviews = 0
cnt_empty_true_sent = 0
sentence_per_review = []
candidate_sentence_num_cnt_per_review = []
# [user-level] Loop for each user
for user_chunk in list(user_item_candidate_sent_ids.items()):
    user_id_str = str(user_chunk[0])
    # assert isinstance(user_chunk[0], str)
    # [item-level] Loop for each user-item pair
    user_item_chunks = list(user_chunk[1].items())
    for item_chunk in user_item_chunks:
        item_id_str = str(item_chunk[0])
        # assert isinstance(item_chunk[0], str)
        candidate_sent_ids = item_chunk[1][0]
        true_sent_ids = item_chunk[1][1]
        if len(true_sent_ids) == 0:
            cnt_empty_true_sent += 1
        else:
            assert isinstance(candidate_sent_ids[0], str)
            assert isinstance(true_sent_ids[0], str)
            # make sure that all true label sent_ids appears in the corresponding candidate set
            for true_sent_id in true_sent_ids:
                assert true_sent_id in candidate_sent_ids
            sentence_per_review.append(len(true_sent_ids))
            candidate_sentence_num_cnt_per_review.append(len(candidate_sent_ids))
        cnt_unique_reviews += 1

print("Total number of unique selected reviews: {}".format(cnt_unique_reviews))
print("Total number of review with empty true sentences: {}".format(cnt_empty_true_sent))
print("Total number of unique review with non-empty true sentences: {}".format(
    cnt_unique_reviews - cnt_empty_true_sent))

Total number of unique selected reviews: 191227
Total number of review with empty true sentences: 0
Total number of unique review with non-empty true sentences: 191227


In [48]:
print("Totally {} user-item pairs in the trainset".format(len(sentence_per_review)))
print("max number of true sentence per review: {}".format(np.max(sentence_per_review)))
print("min number of true sentence per review: {}".format(np.min(sentence_per_review)))
print("mean number of true sentence per review: {}".format(np.mean(sentence_per_review)))
print("max number of candidate sentence per review: {}".format(np.max(candidate_sentence_num_cnt_per_review)))
print("min number of candidate sentence per review: {}".format(np.min(candidate_sentence_num_cnt_per_review)))
print("mean number of candidate sentence per review: {}".format(np.mean(candidate_sentence_num_cnt_per_review)))

Totally 191227 user-item pairs in the trainset
max number of true sentence per review: 23
min number of true sentence per review: 1
mean number of true sentence per review: 2.68899266316995
max number of candidate sentence per review: 520
min number of candidate sentence per review: 32
mean number of candidate sentence per review: 270.1224931625764


In [49]:
useritem2sentids_filepath = '../Dataset/{}/train/useritem2sentids.json'.format(dataset_name)
with open(useritem2sentids_filepath, 'w') as f:
    print("Write file: {}".format(useritem2sentids_filepath))
    json.dump(user_item_candidate_sent_ids, f)

Write file: ../Dataset/yelp/train/useritem2sentids.json


In [51]:
example_user_id = '1001'
example_item_id = '1278'
print("User: {0}\tItem: {1}".format(example_user_id, example_item_id))
print(
    "Number of (sampled) cdd sents: ", 
    len(user_item_candidate_sent_ids[example_user_id][example_item_id][0]))
print("Number of user side cdd sents:", len(user_to_sent[example_user_id]))
print("Number of item side cdd sents:", len(item_to_sent[example_item_id]))
print("Number of GT sentences: ", 
    len(user_item_candidate_sent_ids[example_user_id][example_item_id][1]))

User: 1001	Item: 1278
Number of (sampled) cdd sents:  220
Number of user side cdd sents: 139
Number of item side cdd sents: 84
Number of GT sentences:  3


In [52]:
for sent_id in user_item_candidate_sent_ids[example_user_id][example_item_id][1]:
    assert sent_id in user_item_candidate_sent_ids[example_user_id][example_item_id][0]

## Write train useritem_cdd in a line-by-line format

In [53]:
# Write useritem2sentids into a line-by-line format
train_useritem2sentid_multiline_file = '../Dataset/{}/train/useritem2sentids_multilines.json'.format(dataset_name)
if os.path.exists(train_useritem2sentid_multiline_file):
    print("File: {} exists, remove it.".format(train_useritem2sentid_multiline_file))
    os.remove(train_useritem2sentid_multiline_file)
else:
    print("File: {} doesn't exist, creat it.".format(train_useritem2sentid_multiline_file))

File: ../Dataset/yelp/train/useritem2sentids_multilines.json doesn't exist, creat it.


In [54]:
with open(train_useritem2sentid_multiline_file, 'a') as f1:
    print("Write file: {}".format(train_useritem2sentid_multiline_file))
    cnt_user = 0
    cnt_review = 0
    user_set = set()
    item_set = set()
    useritem_set = set()
    for trainset_user_chunk in list(user_item_candidate_sent_ids.items()):
        assert isinstance(trainset_user_chunk[0], str)
        user_id_str = trainset_user_chunk[0]
        user_id = int(trainset_user_chunk[0])
        user_item_chunks = list(trainset_user_chunk[1].items())
        for item_chunk in user_item_chunks:
            assert isinstance(item_chunk[0], str)
            item_id_str = item_chunk[0]
            item_id = int(item_chunk[0])
            candidate_sent_ids = item_chunk[1][0]
            gold_revw_sent_ids = item_chunk[1][1]
            assert isinstance(candidate_sent_ids, list)
            assert isinstance(gold_revw_sent_ids, list)
            assert isinstance(candidate_sent_ids[0], str)
            assert isinstance(gold_revw_sent_ids[0], str)
            for cur_id in gold_revw_sent_ids:
                assert cur_id in candidate_sent_ids
            cur_data_dict = {'user_id': user_id_str, 'item_id': item_id_str, 'candidate': candidate_sent_ids, "review": gold_revw_sent_ids}
            # write this into the json file
            json.dump(cur_data_dict, f1)
            f1.write("\n")
            cnt_review += 1
            useritem_set.add((user_id_str, item_id_str))
            item_set.add(item_id_str)
            if cnt_review % 50000 == 0:
                print("{} lines of train data written.".format(cnt_review))
        cnt_user += 1
        user_set.add(user_id_str)

assert cnt_user == len(user_set)
assert cnt_review == len(useritem_set)
print("Totally {} users".format(cnt_user))
print("Totally {} items".format(len(item_set)))
print("Totally {} reviews".format(cnt_review))

Write file: ../Dataset/yelp/train/useritem2sentids_multilines.json
50000 lines of train data written.
100000 lines of train data written.
150000 lines of train data written.
200000 lines of train data written.
250000 lines of train data written.
300000 lines of train data written.
350000 lines of train data written.
Totally 9209 users
Totally 7837 items
Totally 390092 reviews


In [55]:
!head -n 1 '../Dataset/yelp/train/useritem2sentids_multilines.json'

{"user_id": "100", "item_id": "10", "candidate": ["863149", "666509", "984394", "833935", "852909", "14132", "8785", "14000", "14059", "81140", "14245", "426338", "13992", "14198", "221648", "14113", "701274", "666508", "763813", "999832", "14189", "704224", "673678", "686221", "101430", "294554", "800076", "14377", "14307", "14317", "629934", "14216", "819225", "661419", "96122", "360284", "14278", "212477", "836672", "913853", "602175", "14009", "14400", "419017", "14370", "956226", "14405", "449351", "714568", "684", "7461", "536603", "838144", "984395", "14103", "567430", "13981", "14380", "166273", "25049", "405317", "13983", "14280", "14133", "247", "14244", "14262", "838143", "381720", "106626", "109570", "109573", "220726", "14116", "55446", "617655", "14217", "14269", "14381", "14253", "14023", "14312", "14291", "653961", "14337", "14051", "14165", "991184", "913852", "389741", "271064", "38318", "541965", "14061", "14210", "753562", "14238", "732857", "178085", "14248", "3514

## Save User-Item Pairs (Train Set)

In [56]:
user_item_pairs = dict()
cnt_user_item_pairs = 0
for trainset_user_chunk in list(user_item_candidate_sent_ids.items()):
    assert isinstance(trainset_user_chunk[0], str)
    user_id_str = trainset_user_chunk[0]
    user_item_chunks = list(trainset_user_chunk[1].items())
    assert user_id_str not in user_item_pairs
    user_item_pairs[user_id_str] = list()
    for item_chunk in user_item_chunks:
        assert isinstance(item_chunk[0], str)
        item_id_str = item_chunk[0]
        assert item_id_str not in user_item_pairs[user_id_str]
        user_item_pairs[user_id_str].append(item_id_str)
        cnt_user_item_pairs += 1
print("Total number of user-item pair on trainset: {}".format(cnt_user_item_pairs))

Total number of user-item pair on trainset: 390092


In [57]:
train_useritempairs_file = '../Dataset/{}/train/useritem_pairs.json'.format(dataset_name)
with open(train_useritempairs_file, 'w') as f:
    print("write file: {}".format(train_useritempairs_file))
    json.dump(user_item_pairs, f)

write file: ../Dataset/yelp/train/useritem_pairs.json
